In [2]:
import pandas as pd  
import numpy as np  
from geopy.distance import distance
import copy
from geopy.distance import geodesic
import matplotlib.pyplot as plt

In [2]:
DATA_SOURCE = 'https://files.data.gouv.fr/arcep_donnees/mobile/sites/2022_T3/2022_T3_sites_Metropole.csv'
print('Link to data source: ', DATA_SOURCE)

Link to data source:  https://files.data.gouv.fr/arcep_donnees/mobile/sites/2022_T3/2022_T3_sites_Metropole.csv


In [3]:
bs_db = pd.read_csv("city_datasets/2022_T3_sites_Metropole.csv")

In [4]:
bs_db.head()

,code_op,nom_op,num_site,id_site_partage,id_station_anfr,x,y,latitude,longitude,nom_reg,...,site_ZB,site_DCC,site_strategique,site_capa_240mbps,date_ouverturecommerciale_5g,site_5g_700_m_hz,site_5g_800_m_hz,site_5g_1800_m_hz,site_5g_2100_m_hz,site_5g_3500_m_hz
0,20801,Orange,00000001A1,NaN,0802290015,687035.0,6985761.0,49.97028,2.81944,Hauts-de-France,...,0,0,0,0,NaN,0,0,0,0,0
1,20801,Orange,00000001B1,NaN,0642290151,422853.0,6249263.0,43.28861,-0.41389,Nouvelle-Aquitaine,...,0,0,0,1,2020-12-14,0,0,0,1,0
2,20801,Orange,00000001B2,NaN,0332290026,416932.0,6422196.0,44.84112,-0.58333,Nouvelle-Aquitaine,...,0,0,0,1,2021-02-22,0,0,0,0,1
3,20801,Orange,00000001B3,NaN,0472290005,511106.0,6349234.0,44.21666,0.63556,Nouvelle-Aquitaine,...,0,0,0,1,NaN,0,0,0,0,0
4,20801,Orange,00000001C1,NaN,0512290147,836824.0,6889450.0,49.09028,4.87333,Grand Est,...,0,0,0,1,NaN,0,0,0,0,0


In [5]:
print("DB size: ", len(bs_db))

DB size:  100212


In [6]:
bs_db.columns

Index(['code_op', 'nom_op', 'num_site', 'id_site_partage', 'id_station_anfr',
       'x', 'y', 'latitude', 'longitude', 'nom_reg', 'nom_dep', 'insee_dep',
       'nom_com', 'insee_com', 'site_2g', 'site_3g', 'site_4g', 'site_5g',
       'mes_4g_trim', 'site_ZB', 'site_DCC', 'site_strategique',
       'site_capa_240mbps', 'date_ouverturecommerciale_5g', 'site_5g_700_m_hz',
       'site_5g_800_m_hz', 'site_5g_1800_m_hz', 'site_5g_2100_m_hz',
       'site_5g_3500_m_hz'],
      dtype='object')

In [7]:
OPERATOR_NAME = 'Orange'
AREA_OF_INTEREST = 'Paris'

In [8]:
bs_db_filtered = bs_db[(bs_db['nom_op'] == OPERATOR_NAME) & (bs_db['nom_dep'] == AREA_OF_INTEREST)]

In [9]:
bs_db_filtered.head()

,code_op,nom_op,num_site,id_site_partage,id_station_anfr,x,y,latitude,longitude,nom_reg,...,site_ZB,site_DCC,site_strategique,site_capa_240mbps,date_ouverturecommerciale_5g,site_5g_700_m_hz,site_5g_800_m_hz,site_5g_1800_m_hz,site_5g_2100_m_hz,site_5g_3500_m_hz
62,20801,Orange,00000002U8,NaN,0752290181,649636.0,6866576.0,48.89722,2.31305,Île-de-France,...,0,0,0,1,NaN,0,0,0,0,0
98,20801,Orange,00000003U7,NaN,0752290285,652147.0,6859914.0,48.83750,2.34805,Île-de-France,...,0,0,0,1,2021-11-29,0,0,0,0,1
99,20801,Orange,00000003U8,NaN,0752290316,650919.0,6866504.0,48.89667,2.33056,Île-de-France,...,0,0,0,1,NaN,0,0,0,0,0
138,20801,Orange,00000004U8,NaN,0752290341,651510.0,6866591.0,48.89750,2.33861,Île-de-France,...,0,0,0,1,2021-03-16,0,0,0,0,1
178,20801,Orange,00000005U7,NaN,0752290284,652922.0,6859939.0,48.83778,2.35861,Île-de-France,...,0,0,0,1,2021-04-30,0,0,0,0,1


In [10]:
# Count the total number of 4G and 5G sites
total_4g_sites = bs_db_filtered['site_4g'].sum()
total_5g_sites = bs_db_filtered['site_5g'].sum()

print(f"Total 4G sites: {total_4g_sites}")
print(f"Total 5G sites: {total_5g_sites}")

Total 4G sites: 734
Total 5G sites: 154


**It is more relevant to analyse 4G sites**

In [11]:
SITE_TYPE = 'site_4g'

In [12]:
#creating the db to be used in the simulator
bs_db_filtered_4g = bs_db_filtered[bs_db_filtered[SITE_TYPE] > 0]
# Select only the required columns
bs_db_filtered_4g = bs_db_filtered_4g[['id_station_anfr', 'latitude', 'longitude']].reset_index(drop=True).rename(columns={'id_station_anfr': 'id'})

In [13]:
bs_db_filtered_4g.head()

,id,latitude,longitude
0,0752290181,48.89722,2.31305
1,0752290285,48.83750,2.34805
2,0752290316,48.89667,2.33056
3,0752290341,48.89750,2.33861
4,0752290284,48.83778,2.35861


### **Grid Perimeter**  
*from Geospatial coordinate to meters*

longitude $\rightarrow$ x (km)

latitude $\rightarrow$ y (km)

In [14]:
min_lat = min(bs_db_filtered_4g['latitude'])
max_lat = max(bs_db_filtered_4g['latitude'])

min_lng = min(bs_db_filtered_4g['longitude'])
max_lng = max(bs_db_filtered_4g['longitude'])

**Grid's reference system**
*based on BSs distributions over the area of interest*

**origin $\rightarrow$ lower-right most point**

**max_coord $\rightarrow$ upper-left most point**

In [15]:
origin = (min_lat,min_lng) #of the grid's reference system
max_coord = (max_lat,max_lng)

In [16]:
#print the grid extema
print("Grid origin geo-coordinate: ", origin)
print("Grid max-point geo-coordinate: ", max_coord)

Grid origin geo-coordinate:  (48.81778, 2.23333)
Grid max-point geo-coordinate:  (48.90139, 2.46501)


In [17]:
bs_db_filtered_4g.head()

,id,latitude,longitude
0,0752290181,48.89722,2.31305
1,0752290285,48.83750,2.34805
2,0752290316,48.89667,2.33056
3,0752290341,48.89750,2.33861
4,0752290284,48.83778,2.35861


In [18]:
#center on the min lat, min lng point as reference
#scale the bs_db to cope with that
#so the coordinate x and y will be in km as the distance from the origin
D_METRIC = 'km'
bs_db_filtered_4g["x"] = [round(
                                distance((min_lat, bs_db_filtered_4g.loc[i].longitude), origin).km, 3) 
                                                                                                    for i in bs_db_filtered_4g.index]


bs_db_filtered_4g["y"] = [round(
                                distance((bs_db_filtered_4g.loc[i].latitude, min_lng), origin).km, 3) 
                                                                                                    for i in bs_db_filtered_4g.index]

In [20]:
#visualization of the db
bs_db_filtered_4g

,id,latitude,longitude,x,y
0,0752290181,48.89722,2.31305,5.855,8.834
1,0752290285,48.83750,2.34805,8.425,2.193
2,0752290316,48.89667,2.33056,7.140,8.773
3,0752290341,48.89750,2.33861,7.732,8.865
4,0752290284,48.83778,2.35861,9.200,2.224
...,...,...,...,...,...
729,0752292366,48.85000,2.32473,6.712,3.583
730,0752292446,48.87556,2.30139,4.998,6.426
731,0752292700,48.84667,2.24888,1.142,3.213
732,0752292729,48.86194,2.30500,5.263,4.911


## Paris Simulation - Parameters

**export the BSs positions, scaled in meters-based reference system**

In [21]:
print("Reference Point:", origin)

Reference Point: (48.81778, 2.23333)


In [22]:
paris_grid = bs_db_filtered_4g[["id","x","y"]]#x,y are in km #redundant cmd add for safety
paris_grid.head()

,id,x,y
0,0752290181,5.855,8.834
1,0752290285,8.425,2.193
2,0752290316,7.140,8.773
3,0752290341,7.732,8.865
4,0752290284,9.200,2.224


In [23]:
# Path to save the CSV file
output_file_path = '../simulation_datasets/Filtered_4G_Sites_Orange_Paris.csv'

paris_grid.to_csv(output_file_path, index=False)

### Grid Dimension

In [24]:
#reference points
base_lft_pt = (min_lat,min_lng)
base_rgt_pt = (min_lat,max_lng)

height_lft_pt = (min_lat,min_lng)
height_rgt_pt = (max_lat,min_lng)

In [25]:
#this it the diagonal in km
diag = round(distance(origin, max_coord).km, 3)
print("diagonal: ", diag, "km")

diagonal:  19.377 km


In [26]:
#so we can build the grid
grid_base = round(distance(base_lft_pt, base_rgt_pt).km, 3)
print("grid_base: ", grid_base, "km")
#this it the base in m

grid_base:  17.014 km


In [27]:
#so we can build the grid
grid_height = round(distance(height_lft_pt, height_rgt_pt).km, 3)
print("grid_height: ", grid_height, "km")
#this it the base in m

grid_height:  9.298 km


In [29]:
#generate geospatial distances based file for the mobility model
out = '../simulation_datasets/Paris_4G_Sites_Lat_Long.csv'

bs_db_filtered_4g[["id","latitude","longitude"]].to_csv(out, index=False)

In [30]:
#full dataset

out = '../simulation_datasets/4G_Site_km.csv'
bs_db_filtered_4g.to_csv(out, index=False)

In [31]:
#full dataset in m
out = '../simulation_datasets/4G_Site_m.csv'
bss = copy.deepcopy(bs_db_filtered_4g)
converter_to_m = lambda v: v*10**3

bss[['x','y']] = bss[['x','y']].map(converter_to_m)

bss.to_csv(out, index=False)

In [34]:
# Define the simulation environment Parameters
content = f"""
DISTANCE_METRIC = '{D_METRIC}'
POWER_METRIC = 'dBm'

earth_radius_km = 6371.0 #km

DATA_SOURCE_LINK = '{DATA_SOURCE}'

ORIGIN = {origin}

NET_WIDTH = {grid_base} #'{D_METRIC}'

NET_HEIGHT = {grid_height} #'{D_METRIC}'

GEO_AREA = '{AREA_OF_INTEREST}'

TILE_SIZE = 25/1000  #'{D_METRIC}'

##Constants

W = 20*(10**6) #(M)hz Channel Bandwidth

N = -125 #dBm/Hz background noise

L_ref = 120 #dBm/1km

ALPHA = 3.76 #lin-scale

BS_P_TX = 46 #dBm

BS_MAX_RANGE = 0.47 #km

EPS_BORDER = 0.2 #km
"""
content +="""

def simulation_params():
    # Get all globals defined in this module
    for name, val in globals().items():
        if not name.startswith('__') and not callable(val):
            print(f"{name}: {val}")

"""

filename = '../simulation_files/simulation_env.py'

with open(filename, 'w') as file:
    file.write(content)